In [ ]:
# Brain Glioma Classification - Depending on the characteristics of the patients, 
#I build a classifier to classify brain tumors in different catefories

In [90]:
import pandas as pd
import numpy as np
import seaborn as sns

/home/carcrupe/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [172]:
df = pd.read_csv('clinical.tsv', sep='\t')

In [173]:
df.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,2573f4bb-998a-4424-b241-3ada0627a92e,C3N-01818,CPTAC-3,'--,'--,'--,'--,'--,-22156,270,...,'--,'--,'--,'--,'--,'--,'--,'--,'--,'--
1,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,'--,'--,'--,'--,-20394,'--,...,'--,'--,'--,'--,'--,'--,'--,not reported,'--,"Pharmaceutical Therapy, NOS"
2,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,'--,'--,'--,'--,-20394,'--,...,'--,'--,'--,'--,'--,'--,'--,not reported,'--,"Radiation Therapy, NOS"
3,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,'--,'--,'--,'--,-20065,320,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Radiation Therapy, NOS"
4,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,'--,'--,'--,'--,-20065,320,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"


In [174]:
print('There are {} rows and {} columns in the clinical data'.format(df.shape[0],df.shape[1]))

There are 2373 rows and 149 columns in the clinical data


In [175]:
# Let us check if we have missing values as NaN
df[df.notna()].shape

(2373, 149)

In [176]:
#It seems we do not have missing values since the shape of the not NAN data frame is equal to the original data frame
#However, above we have seen some not relevant fields that we have to consider as empty
df['age_at_index'][0]

"'--"

In [177]:
#Fill in the not relevant fields with NaN, for posterior fill or drop
df[df == df['age_at_index'][0]] = np.nan

In [178]:
df.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,2573f4bb-998a-4424-b241-3ada0627a92e,C3N-01818,CPTAC-3,NaN,NaN,NaN,NaN,NaN,-22156,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,NaN,NaN,NaN,NaN,-20394,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not reported,NaN,"Pharmaceutical Therapy, NOS"
2,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,NaN,NaN,NaN,NaN,-20394,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not reported,NaN,"Radiation Therapy, NOS"
3,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,NaN,NaN,NaN,NaN,-20065,320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,"Radiation Therapy, NOS"
4,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,NaN,NaN,NaN,NaN,-20065,320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,"Pharmaceutical Therapy, NOS"


In [179]:
#I have filled all empty fields with NaN. I will check now which columns are 50% or more empty and remove them
emptyCols = [col for col in df.columns if df[col].isna().sum() > 0.5*df.shape[0]]
len(emptyCols)

119

In [180]:
df.shape[0]

2373

In [181]:
#Let us remove columns with no data. We have 119 with more that 50% of missing values
df.drop(axis=1, columns=emptyCols, inplace=True)

In [182]:
df.columns

Index(['case_id', 'case_submitter_id', 'project_id', 'age_at_index',
       'days_to_birth', 'days_to_death', 'ethnicity', 'gender', 'race',
       'vital_status', 'year_of_birth', 'age_at_diagnosis',
       'classification_of_tumor', 'days_to_diagnosis',
       'days_to_last_follow_up', 'icd_10_code', 'last_known_disease_status',
       'morphology', 'primary_diagnosis', 'prior_malignancy',
       'prior_treatment', 'progression_or_recurrence',
       'site_of_resection_or_biopsy', 'synchronous_malignancy',
       'tissue_or_organ_of_origin', 'tumor_grade', 'tumor_stage',
       'year_of_diagnosis', 'treatment_or_therapy', 'treatment_type'],
      dtype='object')

In [165]:
# I will check all columns one by one to see if they contain relevant information. If they are candidates to me removed
# add them to the list "not_useful_cols"
not_useful_cols = []

In [166]:
# Cause_of_death column is either Nan or "Cancer related", it does not add any value --> Drop
cause_of_death = df[df['cause_of_death'].notna()]
not_useful_cols.append(cause_of_death)

In [168]:
age_at_index = df[df['age_at_index'].notna()]
age_at_index

,case_id,case_submitter_id,project_id,age_at_index,cause_of_death,days_to_birth,days_to_death,ethnicity,gender,race,...,tumor_largest_dimension_diameter,tumor_stage,year_of_diagnosis,days_to_treatment_end,days_to_treatment_start,initial_disease_status,therapeutic_agents,treatment_intent_type,treatment_or_therapy,treatment_type
1,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,NaN,-20394,NaN,not hispanic or latino,male,black or african american,...,NaN,not reported,2013,NaN,NaN,NaN,NaN,NaN,not reported,"Pharmaceutical Therapy, NOS"
2,ba910760-ebb5-41ba-81a1-c5355e3d3087,TCGA-DU-A7TJ,TCGA-LGG,55,NaN,-20394,NaN,not hispanic or latino,male,black or african american,...,NaN,not reported,2013,NaN,NaN,NaN,NaN,NaN,not reported,"Radiation Therapy, NOS"
3,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,NaN,-20065,320,not hispanic or latino,male,white,...,NaN,not reported,2005,NaN,NaN,NaN,NaN,NaN,yes,"Radiation Therapy, NOS"
4,b584d0a2-d3d2-44b8-85dc-1b3065b499eb,TCGA-02-0039,TCGA-GBM,54,NaN,-20065,320,not hispanic or latino,male,white,...,NaN,not reported,2005,NaN,NaN,NaN,NaN,NaN,yes,"Pharmaceutical Therapy, NOS"
5,f6c6e2c9-a3d6-4b81-a421-04ffef6b8744,TCGA-12-3644,TCGA-GBM,47,NaN,-17468,1818,not hispanic or latino,female,white,...,NaN,not reported,2004,NaN,NaN,NaN,NaN,NaN,yes,"Radiation Therapy, NOS"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,97094aed-69f7-4f4e-a87e-533979e11514,TCGA-12-0829,TCGA-GBM,75,NaN,-27471,626,not hispanic or latino,male,black or african american,...,NaN,not reported,2006,NaN,NaN,NaN,NaN,NaN,yes,"Pharmaceutical Therapy, NOS"
2369,d87742fd-ff94-4ef0-bbf4-5dd5a185a0e7,TCGA-P5-A5ET,TCGA-LGG,27,NaN,-10027,NaN,not hispanic or latino,male,white,...,NaN,not reported,2012,NaN,NaN,NaN,NaN,NaN,not reported,"Pharmaceutical Therapy, NOS"
2370,d87742fd-ff94-4ef0-bbf4-5dd5a185a0e7,TCGA-P5-A5ET,TCGA-LGG,27,NaN,-10027,NaN,not hispanic or latino,male,white,...,NaN,not reported,2012,NaN,NaN,NaN,NaN,NaN,not reported,"Radiation Therapy, NOS"
2371,ab24cdc3-74a1-443f-831f-b7fb3821de6a,TCGA-HT-7616,TCGA-LGG,75,NaN,-27685,7,not hispanic or latino,male,white,...,NaN,not reported,2007,NaN,NaN,NaN,NaN,NaN,no,"Radiation Therapy, NOS"
